In [1]:
import importlib
import ldsc_71 as ld1
import ldsc_72 as ld2
import numpy as np
from numba import jit, njit, prange, vectorize
from helperfuncs import *

# reloading modules
importlib.reload(ld1)
importlib.reload(ld2)

In [3]:
# Reload module after any changes
importlib.reload(ld1)

N = 100
S = np.array([np.array([[5, 0], [0, 5]]),
    np.array([[2, 0], [0, 2]])] * 50 )# 50 = N/2
V = np.identity(2) * 10.0


model = ld2.sibreg(S = S)
model.simdata(V, N)

Warning there is no value for theta. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
No value given for allele frequencies.
Effect Vectors Simulated!


In [4]:
model.solve()

No initial guess provided.
Making a matrix of 0s as the initial estimate
Log Likelihood:  -693.7641795251411
Gradient:  [56.2500753   6.09194231 59.61647609]
Log Likelihood:  -692.9407753548742
Gradient:  [55.85386401  6.0050998  59.17742185]
Log Likelihood:  -689.7064522365798
Gradient:  [54.30919301  5.67104426 57.46799   ]
Log Likelihood:  -677.6514366651893
Gradient:  [48.71511569  4.5226066  51.30790292]
Log Likelihood:  -613.0944890593584
Gradient:  [22.70675156  2.248384   24.05498176]
Log Likelihood:  -583.3593438093
Gradient:  [13.33825915  1.33928761 14.17192559]
Log Likelihood:  -559.6869742174282
Gradient:  [6.92356408 0.74277389 7.40323997]
Log Likelihood: -546.214302134833
Gradient:  [3.58082318 0.42166726 3.86355257]
Log Likelihood:  -538.829965819482
Gradient:  [1.70159394 0.23385191 1.86523914]
Log Likelihood:  -535.6527035673745
Gradient:  [0.7225373  0.12566235 0.81479296]
Log Likelihood:  -534.6546221466832
Gradient:  [0.24572465 0.06045354 0.29227067]
Log Likelihoo

(array([[836.03670005,  55.87330833],
        [ 55.87330833, 902.11137033]]),
 (array([836.03670005,  55.87330833, 902.11137033]),
  534.459927371071,
  {'grad': array([ 2.74189759e-06, -4.03925817e-06, -1.37318924e-06]),
   'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
   'funcalls': 15,
   'nit': 12,
   'warnflag': 0}))

In [93]:
def core_logll_loop(V, N, S, theta, u, r):
    
    Gvec = np.zeros_like(V)
    log_ll = 0
        
    for i in prange(N):


        Si = S[i]
        thetai = theta[i, :]
        ui = u[i]
        ri = r[i]
        

        d, ddash = Si.shape
        assert d == ddash # Each S has to be a square matrix

        # calculate log likelihood
        log_ll += -(d/2) * np.log(2 * np.pi)
        dit_sv = np.linalg.det(Si + ri * V)
        log_ll += -(1/2) * np.log(dit_sv)
        log_ll += -(1/2) * np.trace(np.outer(thetai, thetai) @ np.linalg.inv(Si + ri * V))
        log_ll *= 1/ui

        # calculate gradient
        SV_inv = np.linalg.inv(Si + ri * V)
        G = -(1 / 2) * SV_inv
        G += (1 / 2) * np.dot(SV_inv,np.dot(np.outer(thetai, thetai),SV_inv))
        G *= 1/ui

        Gvec += G
        
    return log_ll, Gvec

In [192]:
@njit
def numba_core_logll_loop(V, N, S, theta, u, r):
    
    Gvec = np.zeros_like(V)
    log_ll = 0
        
    for i in prange(N):


        Si = S[i]
        thetai = theta[i, :]
        ui = u[i]
        ri = r[i]
        

        d, ddash = Si.shape
        assert d == ddash # Each S has to be a square matrix

        # calculate log likelihood
        log_ll += -(d/2) * np.log(2 * np.pi)
        dit_sv = np.linalg.det(Si + ri * V)
        log_ll += -(1/2) * np.log(dit_sv)
        log_ll += -(1/2) * np.trace(np.outer(thetai, thetai) @ np.linalg.inv(Si + ri * V))
        log_ll *= 1/ui

        # calculate gradient
        SV_inv = np.linalg.inv(Si + ri * V)
        G = -(1 / 2) * SV_inv
        G += (1 / 2) * np.dot(SV_inv,np.dot(np.outer(thetai, thetai),SV_inv))
        G *= 1/ui

        Gvec += G
        
    return log_ll, Gvec

In [193]:
def outer_neg_logll_grad(V, theta, S, u, r, loopfunc):
        
    # ============================================ #
    # returns negative log likelihood and negative
    # of the gradient
    # ============================================ #
    
    # Unflatten V into a matrix
    d = S[0].shape[0]
    V = return_to_symmetric(V, d)
    
    N = len(S)

    log_ll, Gvec = loopfunc(V, N, S, theta, u, r)


    Gvec = extract_upper_triangle(Gvec)

    return -log_ll, -Gvec

In [96]:
Vin = extract_upper_triangle(V)

In [105]:
%%timeit 
outer_neg_logll_grad(Vin, model.theta, model.S, model.u, model.r, 
                     loopfunc = core_logll_loop)

13.1 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [195]:
%%timeit 
outer_neg_logll_grad(Vin, model.theta, model.S, model.u, model.r,
                     loopfunc = numba_core_logll_loop)

801 µs ± 35.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
